
# Inputs

`CI_Analyst_-_input1.csv` contains 4 columns: `campaign_name`, `spend`, `impressions`, and
`clicks`. Campaign name consists of a variable number of arguments,
separated with the delimiter `"_"`, but all campaign names contain an
age, cluster and device. Furthermore, the age is always the first
element, the cluster is always the second, and the device always starts
at the third. Some, but not all, campaign names contain a date stamp
following the device, but this is not a part of the device name. For
example, if the campaign name is `31-40_notarget_htc_one_1GB_9114` then
the age is `"31-40"`, the cluster is `"notarget"`, the device is
`"htc_one_1GB"`, and the campaign data is from 9-1-2014.

`CI_Analyst_-_input2.csv` contains 3 columns: `campaign_name`, `actions`, and `object_type`. In
this case, campaign name is always an age, cluster and device, separated
by the delimiter `"_"`, but the order of the elements is variable.
Actions contains a list of dictionaries, where each dictionary contains
an `action_type` and a `value`. For example if a given campaign has the
value `[{action_type:likes,value:12}]`, this means the campaign has 12
likes.

Note, `input1` can have multiple campaigns with same age, cluster and
device (but different dates) while `input2` cannot.

# Output

Using python read in the two input files, map them on `key`, where `key` is
`(age,cluster,device)` and create the following four tables, all with the
same columns, and a totals row:

1.  A table grouped by key

2.  A table grouped by age

3.  A table grouped by cluster

4.  A table grouped by device

Columns:

1.  Spend

2.  Impressions

3.  Storied Engagements = Like + Comment + Share

4.  CPVV = Spend / Video Views if object_type is Video, otherwise = 0

5.  Count = Number of distinct campaign names from input1 per row

Please export your results to XLSX, with one table per tab. Hint: If you
are using pandas, this can be achieved easily with `df.to_excel()`. In
addition to providing your final results please provide well-documented
code, as well as a list of any assumptions made with regards to the data
or process.


In [1]:
# Write your code here
import pandas as pd
import json

In [2]:
input1 = pd.read_csv("CI_Analyst_-_input1.csv")
input2 = pd.read_csv("CI_Analyst_-_input2.csv")

In [3]:
# final_input = pd.merge(input1, input2, on="campaign_name", how="outer")

In [4]:
# This function takes a row of data and splits the campaign name into its constituent parts
def split_compagin_name(row):
    
    # Split the campaign name by underscore
    attr = row.campaign_name.split("_")
    
    # Extract age and cluster from the first two elements of the split campaign name
    age = attr[0]
    cluster = attr[1]
    
    # Check if the last element of the split campaign name is all numeric
    # If it is, assume that it is the date and extract it, along with the device from elements in between
    if attr[-1].isnumeric():
        date = attr[-1]
        device = "_".join(attr[2:-1])
    # If the last element is not all numeric, assume that there is no date and extract the device from elements in between
    else:
        date = None
        device = "_".join(attr[2:])
    
    # Return a pandas series with the extracted age, cluster, device and date
    return pd.Series({"age": age, "cluster": cluster, "device":device, "date":date})


In [5]:
# This function takes a row of data and extracts the number of likes, comments and shares from the actions column
def split_actions(row):
    
    # Create a dictionary with initial values of 0 for each action type
    action_values = {'like': 0, 'comment': 0, 'share': 0}
    
    # Check if the actions column is not empty
    if pd.notna(row['actions']):
        
        # Convert the actions column from JSON to a list of dictionaries
        actions = json.loads(row['actions'])
        
        # Loop through each dictionary in the actions list
        for action in actions:
            
            # Get the action type and check if it is in the action_values dictionary
            action_type = action['action_type']
            if action_type in action_values:
                
                # If the action type is in the action_values dictionary, update its value
                action_values[action_type] = action['value']
    
    # Return a pandas series with the extracted action values
    return pd.Series(action_values)


### Split Compagin Name

In [6]:
final_input1 = input1.copy()
final_input1[["age", "cluster", "device", "date"]] = final_input1.apply(split_compagin_name, axis=1)

In [7]:
final_input1.head()

,campaign_name,spend,impressions,clicks,age,cluster,device,date
0,21-30_cluster1_htc_one_2GB_limited_time_offer_...,250.00,1851,51,21-30,cluster1,htc_one_2GB_limited_time_offer,8114
1,<21_allclusters_iphone_6/6+_Vzn_10114,44.54,2866,1,<21,allclusters,iphone_6/6+_Vzn,10114
2,21-30_notarget_htc_one_1GB,366.52,8658,90,21-30,notarget,htc_one_1GB,None
3,<21_cluster3_htc_one_1GB_8114,427.83,1549,79,<21,cluster3,htc_one_1GB,8114
4,31-40_notarget_iPhone_6/6+_all_other_devices,79.14,6885,98,31-40,notarget,iPhone_6/6+_all_other_devices,None


### Map each action to a spereate column

In [8]:
# JSON requires double quotes around property names
final_input2 = input2.copy()
final_input2['actions'] = final_input2['actions'].str.replace("'", "\"")
final_input2[["like", "comment", "share"]] = final_input2.apply(split_actions, axis=1)

In [9]:
final_input2.head()

,campaign_name,actions,object_type,like,comment,share
0,htc_one_1GB_21-30_allclusters,[],LPP,0,0,0
1,allclusters_21-30_htc_one_2GB,"[{""value"": 17, ""action_type"": ""comment""}]",VIDEO,0,17,0
2,21-30_htc_one_2GB_limited_time_offer_allclusters,[],LPP,0,0,0
3,21-30_iPhone_6/6+_all_other_devices_allclusters,[],VIDEO,0,0,0
4,iphone5S_21-30_allclusters,[],VIDEO,0,0,0


In [10]:
def init_CPVV_engagements(df):
    """
    This function takes a pandas dataframe and initializes two new columns called 'engagements' and 'CPVV'.
    The 'engagements' column is initialized to zero and represents the number of engagements for each row.
    The 'CPVV' column is initialized to zero and represents the cost per video view for each row.
    The function returns the modified dataframe.
    """
    # reset the index of the dataframe
    df.reset_index(inplace=True)
    # initialize the 'engagements' column to zero for all rows
    df[["engagements"]] = 0
    # initialize the 'CPVV' column to zero for all rows
    df[["CPVV"]] = 0.0
    # return the modified dataframe
    return df


In [11]:
def match_names(row1, row2, columns_to_match):
    """
    This function takes two rows from different dataframes and a list of columns to match.
    It returns True if all values in the specified columns of row1 are found in the campaign_name column of row2.
    Otherwise, it returns False.
    """
    # iterate over the columns to match
    for col in columns_to_match:
        # check if the value of the column in row1 is not found in the campaign_name column of row2
        if row1[col] not in row2.campaign_name:
            return False
    # if all values in the specified columns of row1 are found in the campaign_name column of row2, return True
    return True


In [12]:
def calc_engagements(df1, df2, keys):
    """
    Calculates the total engagements for each row in df1 that matches a row in df2 based on the keys specified.

    Args:
    - df1: pandas DataFrame containing the data to be processed
    - df2: pandas DataFrame containing the reference data used for matching
    - keys: list of column names used for matching the two DataFrames

    Returns:
    - pandas DataFrame with a modified "engagements" column containing the calculated total engagements
    """

    # loop through each row in df1
    for i, row1 in df1.iterrows():
        # loop through each row in df2
        for j, row2 in df2.iterrows():
            # check if the rows match based on the specified keys
            if match_names(df1.iloc[i], df2.iloc[j], keys):
                # calculate the total engagements and add it to the "engagements" column in df1
                df1.iloc[i, df1.columns.get_loc("engagements")] = df2.iloc[j, df2.columns.get_loc("like")] + \
                df2.iloc[j, df2.columns.get_loc("comment")] + df2.iloc[j, df2.columns.get_loc("share")]
    # return the modified DataFrame
    return df1


In [28]:
def calc_CPVV(df1, df2, keys):
    """
    Calculates the CPVV (cost per viewable view) for each row in df1 that matches a row in df2 based on the keys specified.

    Args:
    - df1: pandas DataFrame containing the data to be processed
    - df2: pandas DataFrame containing the reference data used for matching
    - keys: list of column names used for matching the two DataFrames

    Returns:
    - pandas DataFrame with a new column "CPVV" containing the calculated CPVV values
    """

    # loop through each row in df1
    for i, row1 in df1.iterrows():
        # loop through each row in df2
        for j, row2 in df2.iterrows():
            # check if the rows match based on the specified keys
            if match_names(df1.iloc[i], df2.iloc[j], keys):
                # if the object type is "VIDEO" and engagements is not 0, calculate CPVV and add it to the CPVV column
                if df2.iloc[j, df2.columns.get_loc("object_type")] == "VIDEO" and df1.iloc[i, df1.columns.get_loc("engagements")] != 0:
                    df1.iloc[i, df1.columns.get_loc("CPVV")] = round(df1.iloc[i, df1.columns.get_loc("spend")] / df1.iloc[i, df1.columns.get_loc("engagements")], 2)
                # if the object type is not "VIDEO" or engagements is 0, set the CPVV value to 0
                else: 
                    df1.iloc[i, df1.columns.get_loc("engagements")] = 0
    # return the modified DataFrame
    return df1


In [40]:
def generate_report(df1, df2, key):
    """
    This function takes two pandas dataframes and a key, which is used to merge the dataframes.
    It generates a report by grouping and aggregating data from the first dataframe, initializing two new columns
    for the resulting dataframe, calculating the number of engagements and cost per video view for each row using the 
    second dataframe, and returning the resulting dataframe.
    """
    # group the data in the first dataframe by age, cluster, and device, and sum the 'spend' and 'impressions' columns
    campaign_by_key = df1.groupby(key).agg({"spend": "sum", "impressions": "sum"})
    # count the number of rows for each group and add the result to a new column called 'count'
    campaign_by_key["count"] = df1.groupby(key).size().values
    # initialize two new columns called 'engagements' and 'CPVV' using the 'initialize_df' function
    campaign_by_key = init_CPVV_engagements(campaign_by_key)
    # calculate the number of engagements for each row using the 'calc_engagements' function
    campaign_by_key = calc_engagements(campaign_by_key, df2, key)
    # calculate the cost per video view for each row using the 'calc_CPVV' function
    campaign_by_key = calc_CPVV(campaign_by_key, df2, key)
    # return the resulting dataframe
    return campaign_by_key


## First table (Age-Device-Cluster) as a key

In [34]:
key1 = ["age", "device", "cluster"]

campaign_by_key1 = generate_report(final_input1, final_input2, key1)

In [36]:
campaign_by_key1.head()

,age,device,cluster,spend,impressions,count,engagements,CPVV
0,21-30,htc_one_1GB,allclusters,1499.98,36378,5,0,0.00
1,21-30,htc_one_1GB,cluster1,969.77,10167,4,21,46.18
2,21-30,htc_one_1GB,cluster2,546.97,15624,5,1,546.97
3,21-30,htc_one_1GB,cluster3,755.73,20997,1,0,0.00
4,21-30,htc_one_1GB,notarget,1405.76,61937,3,16,87.86


## Second table (Age) as a key

In [41]:
key2 = ["age"]

campaign_by_key2 = generate_report(final_input1, final_input2, key2)

In [44]:
campaign_by_key2.head()

,age,spend,impressions,count,engagements,CPVV
0,21-30,43267.84,1035451,186,0,0.00
1,31-40,43154.31,1013886,191,0,2397.46
2,41-50,54252.54,1077556,213,0,0.00
3,51+,45830.80,1017474,189,0,0.00
4,<21,52090.31,1238427,221,0,2083.61


## Second table (cluster) as a key

In [55]:
key3 = ["cluster"]

campaign_by_key3 = generate_report(final_input1, final_input2, key3)

In [56]:
campaign_by_key3.head()

,cluster,spend,impressions,count,engagements,CPVV
0,allclusters,49843.53,1164308,211,0,0.00
1,cluster1,46563.61,1044958,200,0,0.00
2,cluster2,47736.63,1065068,195,0,1193.42
3,cluster3,49199.17,1084853,210,0,0.00
4,notarget,45252.86,1023607,184,0,1810.11


## Second table (device) as a key

In [57]:
key4 = ["device"]

campaign_by_key4 = generate_report(final_input1, final_input2, key4)

In [58]:
campaign_by_key4.head()

,device,spend,impressions,count,engagements,CPVV
0,htc_one_1GB,23796.61,566082,104,0,0.00
1,htc_one_2GB,23354.83,558939,102,0,2123.17
2,htc_one_2GB_limited_time_offer,21258.30,520761,95,0,0.00
3,iPhone_6/6+_all_other_devices,23663.52,540889,97,0,876.43
4,iphone5S,26702.21,497795,92,0,2670.22


### Save to different tabs of the same excel file

In [59]:
writer = pd.ExcelWriter('output.xlsx', engine='xlsxwriter')

# write each dataframe to a different tab in the Excel file
campaign_by_key1.to_excel(writer, sheet_name='By-All')
campaign_by_key2.to_excel(writer, sheet_name='By-Age')
campaign_by_key3.to_excel(writer, sheet_name='By-Cluster')
campaign_by_key4.to_excel(writer, sheet_name='By-Device')

# save the Excel file
writer.save()